In [1]:
"""
Create EOW Services for Web Mapping
"""

# Map Services Extent
ext_shp = '/home/jasp/mrgis/flainar_serv/lmt_pt.shp'

# Tiles width and height
width  = 50000
height = 50000

# Output EPSG
oepsg = 3763

# Layers Definition
layers = {
    'osm_polygons' : (
        "SELECT polygonid, ST_Transform(geom, 3763) AS geom FROM ("
            "SELECT op.polygonid, op.geom, "
            "array_agg(osmcat.key) AS okey, array_agg(osmcat.value) AS ovalue "
            "FROM osm_polygons AS op "
            "INNER JOIN polygons_osmcat AS ocat "
            "ON op.polygonid = ocat.polygoncatid "
            "INNER JOIN osmcategory AS osmcat "
            "ON ocat.osmcategoryid = osmcat.osmcategoryid "
            "GROUP BY op.polygonid, op.geom"
        ") AS foo "
        "WHERE NOT('boundary' = ANY(ovalue))"
    ),
    'osm_points' : (
        "SELECT positionid, name, ST_Transform(geom, 3763) AS geom "
        "FROM osm_position "
        "WHERE osm_id IS NOT NULL"
    )
}

# RefGRID name
refgrid = 'refgrid'

# DB Name
db = 'flainar_gh'

In [2]:
import os
from shapely.wkt import loads
from glass.geo.gt.sample import create_fishnet
from glass.geo.gt.fmshp import shp_to_obj
from glass.sql.to import df_to_db

In [3]:
# Create reference GRID
refgrid_shp = create_fishnet(ext_shp, os.path.join(
    os.path.dirname(ext_shp),
    refgrid + '.shp'
), width, height)

# Remove cells not intersecting with ext_shp
df_grid = shp_to_obj(refgrid_shp, srs_to=oepsg)
df_ref  = shp_to_obj(ext_shp, srs_to=oepsg)

geom = loads(df_ref.loc[0, 'geometry'].wkt)

df_grid['is_int'] = df_grid.geometry.intersects(geom)
df_grid = df_grid[df_grid.is_int == True]

df_grid.reset_index(inplace=True)

df_grid.drop(['index', 'FID', 'is_int'], axis=1, inplace=True)

df_grid['cell_id'] = df_grid.index + 1

# Reference GRID to PostgreSQL
refgrid = df_to_db(db, df_grid, refgrid, colGeom='geometry', epsg=oepsg, geomType='Polygon')

/home/jasp/.virtualenvs/gaspenv/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/jasp/.virtualenvs/gaspenv/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
# Geometries simplification

In [ ]:
base = df_ref.plot(color='white', edgecolor='black')

df_grid.plot(ax=base, color=None, edgecolor='red')